In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect("nominations.db")
q1 = "PRAGMA TABLE_INFO(nominations);"
schema = conn.execute(q1).fetchall()
for i in schema:
    print(i)

(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Category', 'TEXT', 0, None, 0)
(2, 'Nominee', 'TEXT', 0, None, 0)
(3, 'Won', 'INTEGER', 0, None, 0)
(4, 'Movie', 'TEXT', 0, None, 0)
(5, 'Character', 'TEXT', 0, None, 0)


In [3]:
q2 = "SELECT * FROM nominations LIMIT 10;"
first_ten = conn.execute(q2).fetchall()
for i in first_ten:
    print(i)

(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(2010, 'Actor -- Supporting Role', 'Geoffrey Rush', 0, "The King's Speech", 'Lionel Logue')


In [4]:
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]
q_create = "CREATE TABLE ceremonies (id integer PRIMARY KEY, year integer, host text);"
conn.execute(q_create)
q_insert = "INSERT INTO ceremonies (Year, Host) VALUES (?, ?);"
conn.executemany(q_insert, years_hosts)

In [5]:
q_create = "SELECT * FROM ceremonies LIMIT 10;"
conn.execute(q_create).fetchall()

[(1, 2010, 'Steve Martin'),
 (2, 2009, 'Hugh Jackman'),
 (3, 2008, 'Jon Stewart'),
 (4, 2007, 'Ellen DeGeneres'),
 (5, 2006, 'Jon Stewart'),
 (6, 2005, 'Chris Rock'),
 (7, 2004, 'Billy Crystal'),
 (8, 2003, 'Steve Martin'),
 (9, 2002, 'Whoopi Goldberg'),
 (10, 2001, 'Steve Martin')]

In [6]:
conn.execute("PRAGMA foreign_keys = ON;")

In [7]:
q_create2 = '''
CREATE TABLE nominations_two (
id integer PRIMARY KEY,
category text,
nominee text,
movie text,
character text,
won integer,
ceremony_id integer,
FOREIGN KEY(ceremony_id) REFERENCES ceremonies(id));
'''
conn.execute(q_create2)

In [8]:
q_join ='''
SELECT nominations.category, nominations.nominee, nominations.movie, nominations.character, nominations.won, ceremonies.id 
FROM nominations
INNER JOIN ceremonies
ON nominations.year == ceremonies.year
'''
joined_nominations = conn.execute(q_join).fetchall()

In [9]:
q_insert2 = '''
INSERT INTO nominations_two
(category, nominee, movie, character, won, ceremony_id)
VALUES (?, ?, ?, ?, ?, ?)
'''
conn.executemany(q_insert2, joined_nominations)

In [10]:
pd.read_sql_query("SELECT * FROM nominations_two LIMIT 5", conn)

,id,category,nominee,movie,character,won,ceremony_id
0,1,Actor -- Leading Role,Javier Bardem,Biutiful,Uxbal,0,1
1,2,Actor -- Leading Role,Jeff Bridges,True Grit,Rooster Cogburn,0,1
2,3,Actor -- Leading Role,Jesse Eisenberg,The Social Network,Mark Zuckerberg,0,1
3,4,Actor -- Leading Role,Colin Firth,The King's Speech,King George VI,1,1
4,5,Actor -- Leading Role,James Franco,127 Hours,Aron Ralston,0,1


In [11]:
conn.execute("DROP TABLE nominations;")
conn.execute("ALTER TABLE nominations_two RENAME TO nominations;")

In [12]:
create_m = "CREATE TABLE movies (id integer PRIMARY KEY, movie text);"
create_a = "CREATE TABLE actors (id integer PRIMARY KEY, actor text);"
create_ma = '''
CREATE TABLE movies_actors (
id integer PRIMARY KEY, 
movie_id integer, 
actor_id integer,
FOREIGN KEY(movie_id) REFERENCES movies(id),
FOREIGN KEY(actor_id) REFERENCES actors(id)
);'''
conn.execute(create_m)
conn.execute(create_a)
conn.execute(create_ma)

In [13]:
movies = conn.execute("SELECT DISTINCT movie FROM nominations;").fetchall()
actors = conn.execute("SELECT DISTINCT nominee FROM nominations;").fetchall()
insert_m = "INSERT INTO movies (movie) VALUES (?);"
insert_a = "INSERT INTO actors (actor) VALUES (?);"
conn.executemany(insert_m, movies)
conn.executemany(insert_a, actors)

In [14]:
mas = '''
SELECT movies.id, actors.id FROM movies
INNER JOIN nominations ON movies.movie == nominations.movie
INNER JOIN actors ON actors.actor == nominations.nominee;
'''
movies_actors = conn.execute(mas).fetchall()
insert_ma = "INSERT INTO movies_actors (movie_id, actor_id) VALUES (?, ?)"
conn.executemany(insert_ma, movies_actors)

In [15]:
pd.read_sql_query("SELECT * FROM movies LIMIT 5", conn)

,id,movie
0,1,Biutiful
1,2,True Grit
2,3,The Social Network
3,4,The King's Speech
4,5,127 Hours


In [16]:
pd.read_sql_query("SELECT * FROM actors LIMIT 5", conn)

,id,actor
0,1,Javier Bardem
1,2,Jeff Bridges
2,3,Jesse Eisenberg
3,4,Colin Firth
4,5,James Franco


In [17]:
pd.read_sql_query("SELECT * FROM movies_actors LIMIT 5", conn)

,id,movie_id,actor_id
0,1,1,1
1,2,2,19
2,3,2,2
3,4,3,3
4,5,4,4


In [18]:
conn.close()